For simple machine translation, let's take a input sentence "My Name is Pranjal". So, our input sequence will be of length 4. This means there will be 4 vectors representing each word or token of the sentence.

Every single token that is the input for transformer has 3 vectors. They are QUERY, KEY and VALUE.

In [1]:
import numpy as np
import math

## Initializing key, Query and Value vector

In [2]:
# Length of input sequence
length = 4

# Dimension of query, key and value vector
dimension_query = 8
dimension_key = 8
dimension_value = 8

# These dimensions can be treated as hyperparameters that can be tuned

In [3]:
# We first randomly initialize the query, key and value vectors

query = np.random.randn(length, dimension_query)
key = np.random.randn(length, dimension_key)
value = np.random.randn(length, dimension_value)

In [4]:
print(f"Query Vector: {query}")

Query Vector: [[-1.1303875  -1.42801774 -0.71588267  0.58036097  1.19424143 -0.03752522
  -0.63700075 -0.33675827]
 [ 1.57243929  1.32978573 -2.51134871 -0.25950223 -1.23060727  1.19807001
  -1.06043518 -1.02728486]
 [-0.75465409  1.6328243  -0.04977556  2.1744412   0.7167436  -0.29106656
   0.54400683  0.45930289]
 [ 1.90298069  0.03119899 -1.01180199  0.34225403 -1.28214154  1.43511515
  -2.62432109 -1.01178774]]


In [5]:
print(f"Key Vector: {key}")

Key Vector: [[-0.02697205 -2.71193492 -0.38593079  0.81021386  0.69241985  1.89166161
   0.01882549 -0.05619968]
 [ 0.56238015 -0.52521235  0.36790515  0.24937685  0.07838774  0.8157406
   0.33500194 -0.09042501]
 [ 1.2715597  -1.29128152 -0.89002711 -1.00790101 -0.35235882  2.3004613
   1.49676103  0.24458342]
 [-0.7338353   2.16143648  0.63533054  1.05833443 -0.90768088  0.45994196
  -2.04720315  0.24348463]]


In [6]:
print(f"Value Vector: {value}")

Value Vector: [[-0.44509792 -0.11511154  0.58397294  0.95248921  0.46616896 -1.76365161
   0.48395715  0.29531372]
 [ 0.15292593 -0.51082081 -0.35082333 -0.49444306 -0.29816679  0.89694576
   1.39329494 -0.12808377]
 [ 0.63601807 -0.29904161 -0.02258402 -0.49136852  0.31198238 -0.12067309
   0.46660167 -1.36681672]
 [-0.2052244   1.12668762 -0.13763963 -0.35899498 -0.31972739  0.42746004
  -0.93304284  0.78453229]]


In the above examples, the first vector represents "My", second represents "Name" and so on.

## Self Attention

To achieve self attention, we need every single token to look at evry other tokens. This is done by taking a dot product of the QUERY and KEY vectors.

In [7]:
np.matmul(query, key.T)

array([[ 5.412543  , -0.12428535, -1.08410392, -1.97683256],
       [-1.43773379, -0.18426899,  4.1302978 ,  3.43899878],
       [-2.69666978, -0.79857939, -5.21089565,  4.56639237],
       [ 2.36627759,  1.04943133,  2.51279259,  5.34035942]])

From the above random initialization, we can see that the word "My" shows higher affinity with the word "Khadka"

But in the transformer paper, instead of just taking dot product between QUERY and KEY vectors, the product is divided by square root of dimension of KEY vector. This scaling is done to prevent the dot product from becoming too large and stabilize the product.

In [8]:
scaled_product = np.matmul(query, key.T)/math.sqrt(dimension_key)
print(scaled_product)

[[ 1.91362293 -0.04394151 -0.38328862 -0.69891585]
 [-0.50831565 -0.06514892  1.46028079  1.21586968]
 [-0.95341674 -0.28234045 -1.84232983  1.6144635 ]
 [ 0.83660546  0.37103     0.88840634  1.88810218]]


## Masking

In decoder section of the transformer architecture, instead of just self attention block, a masked version of self attention is used. This is because we do not want to look at the future words while generating current word because in real world use, you dont know the words that are going to be generated next. But, masking is not done in encoder since all our inputs are going to be passed simultaneously.

In [9]:
mask = np.tril(np.ones((length, length)))
print(mask)

[[1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]]


This is a simulation of the masking. As we can see, the word "My" can look only at itself, "Name" can look at "My name" and so on.

In [10]:
mask[mask == 0] = - np.infty
mask[mask == 1] = 0

In [16]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [11]:
scaled_product + mask

array([[ 1.91362293,        -inf,        -inf,        -inf],
       [-0.50831565, -0.06514892,        -inf,        -inf],
       [-0.95341674, -0.28234045, -1.84232983,        -inf],
       [ 0.83660546,  0.37103   ,  0.88840634,  1.88810218]])

With this transformation, the lower traingle remains exactly same but upper traingular values become negative infinity and do not provide any value. This negative infinity and zeroes are used since we will be using softmax later.

## Softmax

Softmax function is now used to convert the vectors to probability distribution

In [12]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis = -1)).T

In [14]:
attention_without_mask = softmax(scaled_product)
print(attention_without_mask)

[[0.76038748 0.10736816 0.07647138 0.05577297]
 [0.06524774 0.10163197 0.4672142  0.36590609]
 [0.06095479 0.1192484  0.02505866 0.79473815]
 [0.18041191 0.11325786 0.1900037  0.51632653]]


In [15]:
attention_with_mask = softmax(scaled_product + mask)
print(attention_with_mask)

[[1.         0.         0.         0.        ]
 [0.39098666 0.60901334 0.         0.        ]
 [0.29696113 0.58095746 0.12208142 0.        ]
 [0.18041191 0.11325786 0.1900037  0.51632653]]


So we can see, the softmax converted all negative infinity to zeroes which means that all attention is given to rest of the values in the vector. Lets illustrate this with the first vector representing "My". So, "My" has all its attention on itself only. Similarly, the second vector representing "Name" has its attention on "My" and "Name" with probabilities 0.39 and 0.6.

Now the next step is the dot product between VALUE vector and attention

In [19]:
new_vector_with_better_context = np.matmul(attention_without_mask, value)
new_vector_with_better_context

array([[-0.2848363 , -0.10240467,  0.39697485,  0.61357558,  0.328481  ,
        -1.23014251,  0.50123358,  0.15003407],
       [ 0.20856435,  0.21311888, -0.05846671, -0.34903626,  0.0288856 ,
         0.07611413,  0.04977728, -0.34527989],
       [-0.15605658,  0.81999689, -0.11619257, -0.29852281, -0.25342237,
         0.33615123, -0.53418462,  0.59197415],
       [-0.04809792,  0.44629772, -0.00973587, -0.16287963, -0.05547322,
        -0.01881698, -0.14798548,  0.18414623]])

Finally we have generated a new set of vectors that should represent better context of the words than they were originally using self attention mechanism.

For decoder section, we use the attention mechanish with mask.